# Data Preprocessing & Feature Engineering

## Overview
In this notebook, we will **prepare the dataset** for model training by applying necessary preprocessing steps. Since credit card fraud detection involves **imbalanced data**, proper preprocessing is essential for building an effective model.

## Steps to Cover
1. **Feature Scaling** – Standardizing numerical features to improve model performance.  
2. **Handling Class Imbalance** – Applying **SMOTE (Synthetic Minority Over-sampling Technique)** to balance fraud and non-fraud cases.  
3. **Feature Selection** – Removing redundant or highly correlated features if necessary.  
4. **Final Dataset Check** – Ensuring data is properly preprocessed before moving to model training.

## Why is Preprocessing Important?
- **Fraudulent transactions are rare**, meaning an unbalanced dataset can lead to biased models.
- Some machine learning models (e.g., logistic regression, neural networks) **perform better with scaled features**.
- Oversampling techniques like **SMOTE** help models learn fraud patterns more effectively.

## Next Step: Feature Scaling
Before applying **SMOTE**, we will first scale our features to ensure they are in the same range.  